In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

# Loading environment variables from .env file
load_dotenv()

True

In [2]:
# Getting the Steam API key from the environment variable
api_key = os.getenv('STEAM_API_KEY')
base_url = "https://api.steampowered.com/ISteamNews/GetNewsForApp/v2/"


In [3]:
# Function to fetch news for a given app ID
def fetch_news_for_app(appid, count=10, maxlength=300):
    params = {
        'appid': appid,
        'count': count,
        'maxlength': maxlength,
        'format': 'json'
    }
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        print(f"Failed to retrieve news for app ID {appid}. Status code: {response.status_code}")
        return []
    news_data = response.json()
    return news_data['appnews']['newsitems']


In [4]:
# Define the app IDs for the games you are interested in
games = {
    'The Isle': 376210,
    'ARK: Survival Evolved': 346110,
    'No Man\'s Sky': 275850
}

In [6]:
# Fetch news for each game and store in a dictionary
all_news = {}

for game, appid in games.items():
    print(f"Fetching news for {game}...")
    news_items = fetch_news_for_app(appid)
    all_news[game] = news_items
    print(f"Fetched {len(news_items)} news items for {game}.")

    # Print a sample of the news items for verification
    for item in news_items[:3]:  # Adjust the slice to see more or fewer items
        print(f"Title: {item['title']}\nDate: {item['date']}\nURL: {item['url']}\nContents: {item['contents']}\n")


Fetching news for The Isle...
Fetched 10 news items for The Isle.
Title: DevBlog #49
Date: 1719623867
URL: https://steamstore-a.akamaihd.net/news/externalpost/steam_community_announcements/5842939267315030015
Contents: Filipe - Lead Programmer This month I focused on finishing the grapple mechanic since it will be used as a basis for other upcoming dinosaurs with their own specific variations. Outside of regular bugs, the Hordetest allowed a more “natural” environment to begin balance testing for grapple. Right no...

Title: DevBlog #48
Date: 1717205789
URL: https://steamstore-a.akamaihd.net/news/externalpost/steam_community_announcements/5742731639335599929
Contents: Filipe - Lead Programmer This month I focused on refining the Rex’s crush ability and the new throw it has, which is linked to crush but not exclusive to it. The grapple ability is still receiving some updates, which now is mainly for balance as the grapple values need to align with the entire roste...

Title: Patch 0.14.